## Web Scraping Forbes Global 500 (2015) Data 

In [1]:
import awscli
import sys
import selenium
import unittest
import boto3
import pandas as pd
import tabulate
import time
import requests
import csv
import dataframe
from bs4 import BeautifulSoup
from tabulate import tabulate
from selenium import webdriver

In [2]:
browser = webdriver.Chrome(r"C:\Users\...\chromedriver.exe")
browser.get('https://fortune.com/global500/2015/search/')

In [4]:
#Find EIN filed
inputElement = browser.find_element_by_xpath("//*[@id='content']/div/div[2]/div/div[2]/div/div[2]/span[2]/select")
inputElement.send_keys('100 Rows')

In [5]:
#identify the table to scrape
table = browser.find_element_by_css_selector('div.rt-tbody')

In [6]:
datetime = time.strftime("%Y%m%d-%H%M%S")
filedirectory = 'C:/Users/Jeeho/Documents/Python Scripts/Capstone project/Web_Scrap/'
filename = 'fg500_2015_'
datetime = time.strftime("%Y%m%d%H%M%S")
file = "%s%s%s.csv"%(filedirectory,filename,datetime)

In [7]:
#create empty dataframe Test 2 : Included For Loop for next pages 
data =[]

#Use For Loop for Index

index = 1
while True:
    if index > 5:
        break
    try:
        print("Scraping Page number " + str(index))
        index = index + 1
        for row in table.find_elements_by_css_selector('div.rt-tr'):
            cols = data.append([cell.text for cell in row.find_elements_by_css_selector('div.rt-td.searchResults__cell--2Y7Ce')])

        time.sleep(4)
        next_button = browser.find_element_by_css_selector('div.-next')
        next_button.click()
    except Exception as e:
        print(e)
        break


data1 = pd.DataFrame(data, columns = ['rank','company','revenue % change', 'profits($M)','profits % change','assets($M)','employees'])

data1['year']=2015

print(data1)

browser.quit()

Scraping Page number 1
Scraping Page number 2
Scraping Page number 3
Scraping Page number 4
Scraping Page number 5
    rank                             company revenue % change profits($M)  \
0      1                             Walmart             2.0%      16,363   
1      2                       Sinopec Group            -2.3%       5,177   
2      3                   Royal Dutch Shell            -6.1%      14,874   
3      4            China National Petroleum            -0.8%    16,359.5   
4      5                         Exxon Mobil            -6.1%      32,520   
5      6                                  BP            -9.5%       3,780   
6      7                          State Grid             1.8%     9,796.2   
7      8                          Volkswagen             2.7%    14,571.9   
8      9                        Toyota Motor            -3.4%    19,766.9   
9     10                            Glencore            -5.0%       2,308   
10    11                              

In [9]:
data1.to_csv(file, header=True, line_terminator='\n')

In [ ]:
from botocore.client import Config
from io import StringIO

ACCESS_KEY_ID = 'your_access_key_id'
ACCESS_SECRET_KEY = 'your_access_secret_key'
BUCKET_NAME = 'ceo-culture-sagemaker'

csv_buffer =StringIO()
data1.to_csv(csv_buffer)

s3 = boto3.resource(
        's3',
        aws_access_key_id = ACCESS_KEY_ID,
        aws_secret_access_key=ACCESS_SECRET_KEY,
        config=Config(signature_version='s3v4')
        )
s3.Bucket(BUCKET_NAME).put_object(Key='fg500_2015_20200505022239.csv', Body = csv_buffer.getvalue())

print("Uploaded")